In [ ]:
!unzip ./NIPS_archive.zip

Archive:  ./NIPS_archive.zip
  inflating: authors.csv             
  inflating: papers.csv              


In [ ]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.6 MB/s 
     |████████████████████████████████| 115 kB 85.3 MB/s 
     |████████████████████████████████| 212 kB 92.3 MB/s 
     |████████████████████████████████| 182 kB 86.0 MB/s 
     |████████████████████████████████| 127 kB 86.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.6 MB/s 
     |████████████████████████████████| 7.6 MB 80.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename

In [ ]:
import datasets
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [ ]:
#make hugging face dataset from NIPS papers.csv
df = pd.read_csv('papers.csv', index_col ="title")

In [ ]:
#get rid of all the papers that do NOT have an abstract
for ind in df.index:
  if pd.isnull(df.loc[ind, 'abstract']) or pd.isnull(df.loc[ind, 'full_text']):
    df.drop(ind, inplace = True)

In [ ]:
df

,source_id,year,abstract,full_text
title,,,,
Learning Generative Models with the Up Propagation Algorithm,5220,1997,Up-propagation is an algorithm for inverting ...,Learning Generative Models with the\n\nUp(cid:...
A Neural Network Based Head Tracking System,5221,1997,We have constructed an inexpensive video based...,A Neural Network Based\n\nHead Tracking System...
Algorithms for Non-negative Matrix Factorization,1861,2000,Non-negative matrix factorization (NMF) has pr...,Algorithms for Non-negative Matrix \n\nFactori...
Characterizing Neural Gain Control using Spike-triggered Covariance,1975,2001,Spike-triggered averaging techniques are effec...,Characterizing neural gain control using\n\nsp...
Compressed Regression,195,2007,Recent research has studied the role of sparsi...,Compressed Regression\n\nShuheng Zhou∗ John La...
...,...,...,...,...
Discrete Object Generation with Reversible Inductive Construction,5452,2019,The success of generative modeling in continuo...,Discrete Object Generation\n\nwith Reversible ...
Adaptively Aligned Image Captioning via Adaptive Attention Time,4799,2019,Recent neural models for image captioning usua...,Adaptively Aligned Image Captioning via\n\nAda...
Fully Dynamic Consistent Facility Location,1827,2019,We consider classic clustering problems in ful...,Fully Dynamic Consistent Facility Location\n\n...


In [ ]:
#split the dataframe into sets
train_set, test_set = train_test_split(df, test_size=0.2, random_state=0)


train = Dataset.from_pandas(train_set)
test = Dataset.from_pandas(test_set)

dataset_tuning = DatasetDict()
 
dataset_tuning['train'] = train
dataset_tuning['test'] = test

dataset_tuning

DatasetDict({
    train: Dataset({
        features: ['source_id', 'year', 'abstract', 'full_text', 'title'],
        num_rows: 5088
    })
    test: Dataset({
        features: ['source_id', 'year', 'abstract', 'full_text', 'title'],
        num_rows: 1272
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import create_optimizer, AdamWeightDecay
from transformers import AutoConfig
from transformers import T5Model
import nltk
import numpy as np
nltk.download('punkt')

metric = load_metric("rouge")
tokenizer = 0
prefix = "summarize: "
max_input_length = 1024
max_target_length = 128


#adds padding to input before traing the model on the dataset
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["full_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



#tokenizer for the dataset
def my_tokenize(model_checkpoint):
  global tokenizer

  sum = dataset_tuning

  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  tokenized_sum = sum.map(preprocess_function, batched=True)

  return (tokenizer, tokenized_sum)


#create new summerization model
def get_model(model_checkpoint, tokenizer):
  #make a model that is not pre-trained
  model = AutoModelForSeq2SeqLM.from_pretrained("Dagar/t5-small-science-papers")

  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
  return (model, data_collator)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}


#set hyper paramaters
#change hyper paramters for better trained model
def get_my_hyper_params(model_checkpoint, my_epochs, floating_point):
  batch_size = 16
  model_name = model_checkpoint
  args = Seq2SeqTrainingArguments(
      f"{model_name}-science-papers-NIPS",
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      weight_decay=0.01,
      save_total_limit=20,
      num_train_epochs=my_epochs,
      predict_with_generate=True,
      fp16=floating_point,
      push_to_hub=True,
  )

  return args

#make the trainer
def get_trainer(model, tokenizer, tokenized_sum, data_collator, training_args):
  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sum["train"],
    eval_dataset=tokenized_sum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
	)
  return trainer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  del sys.path[0]


In [ ]:
def my_train_model():
  model_name = "t5-small" #this is used for the default tokenizer
  epochs = 10
  floating_point = True

  token_tuple = my_tokenize(model_name)

  model_tuple = get_model(model_name, token_tuple[0])

  params = get_my_hyper_params(model_name, epochs, floating_point)

  trainer = get_trainer(model_tuple[0], token_tuple[0], token_tuple[1], model_tuple[1], params)

  trainer.train()

  return trainer

In [ ]:
trainer = my_train_model()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


  0%|          | 0/6 [00:00<?, ?ba/s]

...

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/Dagar/t5-small-science-papers-NIPS into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: year, full_text, abstract, title, source_id. If year, full_text, abstract, title, source_id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to 

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,5.185558,13.717200,2.064400,10.218900,12.838000,19.000000
2,5.452200,5.038340,15.621100,2.180800,11.356100,14.305400,19.000000
3,5.452200,4.948597,15.165900,2.330800,11.105200,13.945600,19.000000
4,5.125400,4.885084,15.716000,2.409900,11.495400,14.509900,19.000000
5,4.979400,4.845583,15.550700,2.426700,11.386700,14.323700,19.000000
6,4.979400,4.807345,15.840600,2.425400,11.687800,14.615400,19.000000
7,4.882300,4.787161,15.555400,2.463700,11.340100,14.318300,19.000000
8,4.833800,4.768018,15.478300,2.488800,11.336400,14.203100,19.000000
9,4.833800,4.762086,15.958000,2.566200,11.613900,14.657600,19.000000
10,4.783800,4.756572,15.706600,2.565400,11.467900,14.401700,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: year, full_text, abstract, title, source_id. If year, full_text, abstract, title, source_id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1272
  Batch size = 16
Saving model checkpoint to t5-small-science-papers-NIPS/checkpoint-500
Configuration saved in t5-small-science-papers-NIPS/checkpoint-500/config.json
Model weights saved in t5-small-science-papers-NIPS/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-science-papers-NIPS/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-science-papers-NIPS/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-science-papers-NIPS/tokenizer_config.json
Special tokens file saved in t5-small-science-papers-NIPS/special_tokens_map.json
The fo

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-science-papers-NIPS
Configuration saved in t5-small-science-papers-NIPS/config.json
Model weights saved in t5-small-science-papers-NIPS/pytorch_model.bin
tokenizer config file saved in t5-small-science-papers-NIPS/tokenizer_config.json
Special tokens file saved in t5-small-science-papers-NIPS/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/231M [00:00<?, ?B/s]

Upload file runs/Nov28_18-00-24_5ca121f841d6/events.out.tfevents.1669658434.5ca121f841d6.107.0:  30%|###      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Dagar/t5-small-science-papers-NIPS
   5136336..9d1b2b8  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Dagar/t5-small-science-papers-NIPS
   5136336..9d1b2b8  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 15.7066}]}
To https://huggingface.co/Dagar/t5-small-science-papers-NIPS
   9d1b2b8..066d2d0  main -> main

   9d1b2b8..066d2d0  main -> main



'https://huggingface.co/Dagar/t5-small-science-papers-NIPS/commit/9d1b2b85e1e4e548f84423820d170e661fdc9675'